# 네이버 뉴스 기반 QA 챗봇

In [14]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [15]:
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [16]:
# 1단계: 뉴스기사 내용을 로드
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 직원들의 세금 부담을 고려해 정부가 면세해달라는 제안도 나왔습니다.이같은 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아

In [17]:
# 2단계: Text Splitter 정의 및 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

In [18]:
# 3단계: 임베딩
embedding = HuggingFaceEmbeddings()

# 4단계: 벡터DB에 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

# 5단계: retriever 생성
retrievr = vectorstore.as_retriever()

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
from langchain_core.prompts import PromptTemplate

# 6단계: Prompt 생성
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

In [20]:
prompt_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
)
prompt_chain.invoke("Summarize")

StringPromptValue(text='당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.\n검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.\n한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don\'t narrate the answer, just answer the question. Let\'s think step-by-step.\n\n#Question: \nSummarize \n\n#Context: \n[Document(page_content="[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 \'황당 공약\'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간

In [33]:
# 7단계: LLM 정의
llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    max_new_tokens=2048,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
    # streaming=True,
    # callbacks=[StreamingStdOutCallbackHandler()],
)
model = ChatHuggingFace(llm=llm)

# 8단계: chain 정의
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# class StreamChain:
#     def __init__(self, chain):
#         self.chain = chain

#     def stream(self, query):
#         response = self.chain.stream(query)
#         complete_response = ""
#         for token in response:
#             print(token, end="", flush=True)
#             complete_response += token
#         return complete_response


# 생성자에 chain 을 매개변수로 전달하여 chain 객체를 생성합니다.
# chain = StreamChain(rag_chain)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
answer = rag_chain.invoke("부영그룹의 출산 장려 정책에 대해 설명해주세요.")
answer

'부영그룹은 2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다. 또한 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.'

In [35]:
print(rag_chain.invoke("부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"))

부영그룹은 출산 직원에게 1억원을 지원합니다.


In [36]:
print(rag_chain.invoke("정부의 저출생 대책을 bullet points 형식으로 작성해 주세요."))

정부의 저출생 대책은 다음과 같습니다.

• 매달 주는 부모 급여: 0세 아이는 100만원으로 올렸습니다.
• 첫만남이용권, 아동수당: 아이 돌까지 1년 동안 1520만원을 받습니다.
• 지자체 지원: 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다고 했습니다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다.
• 기업 지원: 한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다. 2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.


In [37]:
print(rag_chain.invoke("부영그룹의 임직원 숫자는 몇명인가요?"))

부영그룹의 임직원 숫자는 주어지지 않았습니다.
